## Imports

In [1]:
DEV = True

import pandas as pd
import numpy as np

import os
import json
import time
import datetime

from os import listdir
from os.path import isfile
from tqdm import tqdm

import warnings
warnings.filterwarnings("ignore")

import matplotlib.pyplot as plt
%matplotlib inline

import lightgbm as lgb
import catboost as cb

from sklearn.model_selection import StratifiedKFold, train_test_split, ParameterSampler
from sklearn.metrics import f1_score
from sklearn.svm import SVC
from sklearn.metrics import roc_auc_score as auc
from sklearn.metrics import fbeta_score, cohen_kappa_score
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
from sklearn.externals import joblib
from sklearn.preprocessing import scale

from bayes_opt import BayesianOptimization
from bayes_opt.observer import JSONLogger
from bayes_opt.event import Events
from bayes_opt.util import load_logs

def lgb_f1_score(y_hat, data):
    y_true = data.get_label()
    y_hat = np.round(y_hat) # scikits f1 doesn't like probabilities
    return 'f1', f1_score(y_true, y_hat), True

def ensure_dir(file_path):
    directory = os.path.dirname(file_path)
    if not os.path.exists(directory):
        os.makedirs(directory)
        
input_path = "../input/"
output_path = "../output/"
model_path = "../models/"

ensure_dir(model_path)
ensure_dir(output_path)

## Read data

In [2]:
train_df = pd.read_csv(input_path+'train.csv.zip')

label = train_df.target
train = train_df.drop(['ID_code','target'],axis=1)
#train = train_df.drop(['ID_code'],axis=1)

test = pd.read_csv(input_path+'test.csv.zip')
test = test.drop(['ID_code'],axis=1)

test_filtered = pd.read_pickle(input_path+'test_filtered.pkl')
test_filtered = test_filtered.loc[:,train.columns]

train_test = pd.concat([train,test_filtered]).reset_index(drop=True)
#train_test.target.fillna(-1, inplace=True)

In [ ]:
train_df.tail()

In [ ]:
# for i in tqdm(range(200)):
#     col = "var_" + str(i) + "_train_test_sum_vcs_prod"
#     train_test[["target", col]].sort_values(by=col).to_csv(output_path+col+".csv", index=False)
#     break

In [3]:
vcs_train_test = {}
mean_target = {}

skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
for train_index, valid_index in skf.split(train_df, train_df.target):
    break
    
train_tmp = train_df.loc[train_index]

for i in tqdm(range(200)):
    col = "var_"+str(i)
    vcs_train_test[col] = train_test.loc[:,col].value_counts()
    
#     train_tmp[col+"_round"] = train_tmp[col].apply(lambda x: round(x,0))
    
#     d0=dict(train_tmp.loc[train_tmp.target==0][col+"_round"].value_counts())
#     d1=dict(train_tmp.loc[train_tmp.target==1][col+"_round"].value_counts())
    
#     mean_target[col] = dict()
#     for k in sorted(list(set(d0)&set(d1))):
#         mean_target[col][k] = d1[k]/d0[k]
              
#     vtraintest = vcs_train_test[col]
#     t = vtraintest[train_test[col]].values
#     train_test[col+'_train_test_sum_vcs'] = t
#     train_test[col+'_train_test_sum_vcs_prod'] = train_test[col]*t
#     vcs_train_test_prod[col] = train_test.loc[:,col+'_train_test_sum_vcs_prod'].value_counts()

100%|██████████| 200/200 [00:03<00:00, 53.21it/s]


In [4]:
def feature_generator(df):
    for i in tqdm(range(200)):
        col = "var_"+str(i)
        
        vtraintest = vcs_train_test[col]
        t = vtraintest[df[col]].values #fillna(0).values
        df[col+'_train_test_sum_vcs'] = t
        df[col+'_train_test_sum_vcs_prod'] = df[col] * t
        
#         t = df[col].apply(lambda x: round(x,0)).apply(lambda x: mean_target[col].get(x,0.11)).values
#         df[col+'_train_test_mean_target'] = t

#         df[col+'_diff'] = df[col]-np.median(df[col])
#         vtraintest_prod = vcs_train_test_prod[col]
#         t = vtraintest_prod[df[col]].values #fillna(0).values
#         df[col+'_train_test_sum_vcs_'] = t
#         df[col+'_train_test_sum_vcs_prod_'] = df[col]*t
        
#         df[col+'_train_test_sum_vcs_sign'] = (df[col+"_train_test_sum_vcs_prod"]>0).astype(int)
#         df[col+'_train_test_sum_vcs_div'] = df[col]/t
#         df[col+'_train_test_sum_vcs_minus'] = scale(df[col]) - scale(t)
#         df[col+'_train_test_sum_vcs_plus'] = scale(df[col]) + scale(t)
#         df[col+'_train_test_sum_vcs_min'] = np.min(scale(df[col]), scale(t))
#         df[col+'_train_test_sum_vcs_max'] = np.max(scale(df[col]), scale(t))
#         df[col+'_train_test_sum_vcs_pow'] = np.power(abs(scale(df[col])), abs(scale(t)))
#         df[col+'_train_test_sum_vcs_log'] = np.log(abs(scale(df[col])), abs(scale(t)))
        
feature_generator(train)
feature_generator(test)

100%|██████████| 200/200 [00:04<00:00, 29.48it/s]


In [5]:
df = train
dt = test

cols = sorted(list(set(df.columns)-{"label"}))
print(len(cols))

df["label"] = label
df.label.value_counts()

600


0    179902
1     20098
Name: label, dtype: int64

In [ ]:
train.var_0_train_test_mean_target.hist(bins=10)

In [ ]:
train.head()

In [ ]:
n = 12

train[f"var_{n}_round"] = train[f"var_{n}"].apply(lambda x: round(x,1))
print(len(train[f"var_{n}_round"].value_counts()))

col = f"var_{n}_round"

d0=dict(train[train.label==0][col].value_counts())
d1=dict(train[train.label==1][col].value_counts())

x = []
y = []
for k in sorted(list(set(d0)&set(d1))):
    res = d1[k]/d0[k]
    #print(k, res, d0[k])
    x.append(k)
    y.append(res)
    
plt.plot(x,y)

## Scaling

In [6]:
scaler = StandardScaler() #MinMaxScaler StandardScaler RobustScaler

X = scaler.fit_transform(df[cols])
#Z = scaler.transform(dt[cols])
y = list(df.label)

skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

## Logistic regression

In [8]:
%%time

df["clf"] = 0
dt["clf"] = 0

i = -1
for train_index, valid_index in skf.split(X, y):
    i+=1
        
    X_train = X[train_index, :]
    X_valid = X[valid_index, :]

    y_train = df.loc[train_index, "label"]
    y_valid = df.loc[valid_index, "label"]
    
    fname = model_path+'clf_'+str(i)+'.pkl'
    if DEV or not isfile(fname):
        c = 10
        clf = LogisticRegression(C=c,
                                 solver="newton-cg",
                                 penalty="l2", 
                                 n_jobs=-1, 
                                 max_iter=100).fit(X_train, y_train) 
        joblib.dump(clf, fname)
    else:
        clf = joblib.load(fname)

    y_pred = clf.predict_proba(X_valid)[:,1] 
    df.loc[valid_index, "clf"] = y_pred
    print("ROC AUC:", round(auc(y_valid, y_pred), 4), 
          "\nF-score:", round(fbeta_score(y_valid, y_pred>0.5, beta=1), 4), i)
    
    #dt["clf"] += clf.predict(Z)/5
    break
    

# print("\nROC AUC:", round(auc(df.label, df["clf"]), 4), 
#       "\nF-score:", round(fbeta_score(df.label, df["clf"]>0.5, beta=1), 4))

ROC AUC: 0.876 
F-score: 0.4295 0
CPU times: user 1.45 s, sys: 5.89 s, total: 7.34 s
Wall time: 4min 44s


## SVM

In [ ]:
#cols+=["clf"]

scaler = StandardScaler() #MinMaxScaler StandardScaler RobustScaler

X = scaler.fit_transform(df[cols])
Z = scaler.transform(dt[cols])
y = list(df.label)

skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

In [ ]:
%%time

df["svc"] = 0
dt["svc"] = 0

i = -1
for train_index, valid_index in skf.split(X, y):
    i+=1
        
    X_train = X[train_index, :]
    X_valid = X[valid_index, :]

    y_train = df.loc[train_index, "label"]
    y_valid = df.loc[valid_index, "label"]
    
    fname = model_path+'svc_'+str(i)+'.pkl'
    #print(fname)
    if DEV or not isfile(fname):
        c = 100
        svc = SVC(C=c,
                  probability=True).fit(X_train, y_train) 
        
        joblib.dump(svc, fname)
    else:
        svc = joblib.load(fname)

    y_pred = svc.predict_proba(X_valid)[:,1] 
    df.loc[valid_index, "svc"] = y_pred
    print("ROC AUC:", round(auc(y_valid, y_pred), 4), 
          "\nF-score:", round(fbeta_score(y_valid, y_pred>0.5, beta=1), 4), i)
    
    #dt["svc"] += svc.predict(Z)/5
    break


# print("\nROC AUC:", round(auc(df.label, df["svc"]), 4), 
#       "\nF-score:", round(fbeta_score(df.label, df["svc"]>0.5, beta=1), 4))

## Nearest Neighbors

In [9]:
#cols+=["svc"]

scaler = StandardScaler() #MinMaxScaler StandardScaler RobustScaler

X = scaler.fit_transform(df[cols])
#Z = scaler.transform(dt[cols])
y = list(df.label)

skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

In [ ]:
%%time

df["nei"] = 0
dt["nei"] = 0

i = -1
for train_index, valid_index in skf.split(X, y):
    i+=1
        
    X_train = X[train_index, :]
    X_valid = X[valid_index, :]

    y_train = df.loc[train_index, "label"]
    y_valid = df.loc[valid_index, "label"]
    
    fname = model_path+'nei_'+str(i)+'.pkl'
    if DEV or not isfile(fname):
        nei = KNeighborsClassifier(n_neighbors=2,
                                   p=1,n_jobs=-1).fit(X_train, y_train) 
        joblib.dump(nei, fname)
    else:
        nei = joblib.load(fname)

    y_pred = nei.predict_proba(X_valid)[:,1] 
    df.loc[valid_index, "nei"] = y_pred
    print("ROC AUC:", round(auc(y_valid, y_pred), 4), 
          "\nF-score:", round(fbeta_score(y_valid, y_pred>0.5, beta=1), 4), i)
    
    #dt["nei"] += nei.predict(Z)/5
    break

# print("\nROC AUC:", round(auc(df.label, df["nei"]), 4), 
#       "\nF-score:", round(fbeta_score(df.label, df["nei"]>0.5, beta=1), 4))

## LighGBM

In [ ]:
#cols+=["nei"]

# k = 100
# cols = [f"var_{k}",
#         # f"var_{k}_train_test_sum_vcs",
#         # f"var_{k}_train_test_sum_vcs_prod"
#        ]

scaler = StandardScaler() #MinMaxScaler StandardScaler RobustScaler

X = df[cols].values #scaler.fit_transform(df[cols])
#Z = dt[cols # ]scaler.transform(dt[cols])
y = list(df.label)

skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

In [ ]:
rounds = 10000
early_stop_rounds = 200

params = {'lambda_l1': 0, 
          'lambda_l2': 0,
          'feature_fraction':0.9,
#           'bagging_fraction':0.1,
#           'bagging_freq': 1,
          'learning_rate': 0.03, 
          'max_depth': 4,
          #'min_data_in_leaf':20,
          #'num_leaves':2**5-1,
          'boosting_type': 'gbrt', #dart gbrt
          'objective': 'binary', 
          'metric': 'auc',
          #'weight': [1, 0.5],
          #'device': 'gpu',
          #'gpu_platform_id': '0',
          #'gpu_device_id': '0',
          'max_bin': 1024,
          'n_jobs':-1
         }

In [ ]:
%%time

df["lgb1"] = 0
dt["lgb"] = 0

i = -1
for train_index, valid_index in skf.split(X, y):
    i+=1
        
    X_train = X[train_index, :]
    X_valid = X[valid_index, :]

    y_train = df.loc[train_index, "label"]
    y_valid = df.loc[valid_index, "label"]

    d_train = lgb.Dataset(X_train, y_train, feature_name=cols)
    d_valid = lgb.Dataset(X_valid, y_valid, feature_name=cols)    
    
    fname = model_path+'lgb_'+str(i)+'.pkl'
    if DEV or not isfile(fname):
        model = lgb.train(params,
                            d_train,
                            num_boost_round=rounds,
                            valid_sets=[d_train, d_valid],
                            valid_names=['train','valid'],
                            #feval=lgb_f1_score,
                            early_stopping_rounds=early_stop_rounds,
                            verbose_eval=100)
        
        joblib.dump(model, fname)
    else:
        model = joblib.load(fname)
    
    
    y_pred = model.predict(X_valid)
    df.loc[valid_index, "lgb1"] = y_pred
    print("ROC AUC:", round(auc(y_valid, y_pred), 4), 
      "\nF-score:", round(fbeta_score(y_valid, y_pred>0.5, beta=1), 4), i)
    
    #dt["lgb"] += model.predict(Z)/5
    break
    
# print("\nROC AUC:", round(auc(df.label, df["lgb"]), 4), 
#       "\nF-score:", round(fbeta_score(df.label, df["lgb"]>0.5, beta=1), 4))

# [3992]	train's auc: 0.989264	valid's auc: 0.913532
# ROC AUC: 0.9135 
# F-score: 0.5195 0

In [ ]:
df.loc[valid_index, "lgb"][df.label==1][df.lgb>0.1]

In [ ]:
w = [1,2]

df["avg"] = (w[0]*df["lgb"]+w[1]*df["lgb1"])/sum(w)

print("\nROC AUC:", round(auc(y_valid, df.loc[valid_index, "avg"]), 4))
print("\nROC AUC:", round(auc(y_valid, df.loc[valid_index, "lgb"]), 4))
print("\nROC AUC:", round(auc(y_valid, df.loc[valid_index, "lgb1"]), 4))

In [ ]:
np.sum(dt["avg"] > 0.5) / len(dt), np.sum(df["avg"] > 0.5) / len(df)

In [ ]:
predictions = list(dt["avg"] > 0.5)
np.mean(predictions[:2*len(predictions)//5])

In [ ]:
dt[["Image","avg"]].to_csv("test_prediction_top5.csv", index=False)

In [ ]:
fig, ax = plt.subplots(figsize=(15, 300))
lgb.plot_importance(model, max_num_features=len(cols), ax=ax)
plt.title("Light GBM Feature Importance")

## False positive

In [ ]:
df[(df["lgb"]>0.5)&(df.label!=1)]

## False negative

In [ ]:
df[(df["lgb"]<0.5)&(df.label==1)]

## Correlation

In [ ]:
df.corr().label

## HPO

In [ ]:
scaler = StandardScaler() #MinMaxScaler StandardScaler RobustScaler

X = scaler.fit_transform(df[cols])
Z = scaler.transform(dt[cols])
y = list(df.label)

skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

In [ ]:
param_grid = {  
              'max_depth': [3,4,5],
              'learning_rate': np.linspace(0.01, 0.1, 10),
              'feature_fraction': np.linspace(0.5, 0.9, 5),
              'lambda_l1': np.linspace(1, 10, 10),
              'lambda_l2': np.linspace(1, 10, 10)
            }

param_static = {
                'boosting_type': 'gbrt',
                'objective': 'binary',
                'metric': 'auc',
                'max_bin': 1024,
                'n_jobs': -1
               }

param_list = list(ParameterSampler(param_grid, 
                                   n_iter=10))

rounded_list = [dict((k, round(v, 6)) for (k, v) in d.items())
                for d in param_list]

rounds = 10000
early_stop_rounds = 200

In [ ]:
for train_index, valid_index in skf.split(X, y):
    X_train = X[train_index, :]
    X_valid = X[valid_index, :]

    y_train = df.loc[train_index, "label"]
    y_valid = df.loc[valid_index, "label"]

    d_train = lgb.Dataset(X_train, y_train, feature_name=cols)
    d_valid = lgb.Dataset(X_valid, y_valid, feature_name=cols)    
    
    break

results = []
for i, params in enumerate(rounded_list):
    print(params)
    start_time = datetime.datetime.fromtimestamp(time.time())
    
    for key in param_static:
        params[key] = param_static[key]
    
    model = lgb.train(params,
                      d_train,
                      num_boost_round=rounds,
                      valid_sets=[d_train, d_valid],
                      valid_names=['train','valid'],
                      #feval=lgb_f1_score,
                      early_stopping_rounds=early_stop_rounds,
                      verbose_eval=0)

    y_pred = model.predict(X_valid)
    
    end_time = datetime.datetime.fromtimestamp(time.time())
    seconds = (end_time - start_time).total_seconds()
    minutes = round(seconds/60, 1)
    
    res = auc(y_valid, y_pred)
    print(i, "ROC AUC:", round(res, 4), "F-score:", 
          round(fbeta_score(y_valid, y_pred>0.5, beta=1), 4), 
          minutes, "minutes\n")
    
    results.append((params, res))
    for key in param_static:
        del params[key]

In [ ]:
sorted_results = sorted(results, key=lambda x: x[1], reverse=True)
sorted_results[0]